In [14]:
import numpy as np
from matplotlib import pyplot as plt

def V(x):
    return x * 0

def density(psi):
    return psi * np.conj(psi)

def gen_gaussian(x,x0,sigma, p0):
    A = (1 / (2 * np.pi * sigma**2))**(0.25)
    B = np.exp((-(x-x0)**2) / ((4 * sigma)**2))
    C = np.exp(1j*p0*x)
    return A * B * C

def normal(x,x0,sigma, p0):
    A = (1 / (2 * np.pi * sigma**2))**(0.25)
    B = np.exp((-(x-x0)**2) / ((4 * sigma)**2))
    C = np.exp(1j*p0*x)
    return A * B * C

def get_psi(psi,V):
    psi_next = np.zeros(N,dtype='cfloat')
    for i in range(1,len(psi)-1):
        # print(f'{psi[i]},{1j/2 * dt/dx**2 * (psi[i+1] - 2*psi[i] + psi[i-1])},{1j*dt*V(psi[i])*psi[i]}')
        psi_next[i] = psi[i] + (1j/2 * dt/dx**2 * (psi[i+1] - 2*psi[i] + psi[i-1])) - (1j*dt*V[i]*psi[i]) 
    return psi_next

In [20]:
N=300
x,dx = np.linspace(-30,30, N , retstep= True)
x.astype(complex)
dt = 1e-7

psi0 = gen_gaussian(x,x0 = 0 ,sigma= 2,p0=2)
psi1 = normal(x,x0 = 0 ,sigma= 2,p0= 0 )

psi00 = psi0.copy()
psi11 = psi1.copy()

for _ in range(10):
    psi00 = get_psi(psi00,V(x))
    psi11 = get_psi(psi11,V(x))

a = np.vstack((psi00,psi11))



In [26]:

# print(psi00 - psi11)


# print(density(psi0) - density(psi1) < 1e-7)

# plt.plot(x,density(psi0) , 'r')
# plt.plot(x,psi0.real , 'g')
# plt.plot(x,psi0.imag , 'b')
# plt.show()
# plt.plot(x,density(psi1))
# plt.plot(x,psi1.real ,)
# plt.plot(x,psi1.imag ,)
# plt.show()

# alist = np.array([1,2,3,4,5,6,7,8,9])
# blist = np.array([2,3,4,5,6,7,8,9,10])
# ablist = np.vstack((alist,blist))
# print(ablist[1,:])




[ 2  3  4  5  6  7  8  9 10]
